In [1]:
# Imports you'll need.
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
import pickle 
import re 
from collections import defaultdict
import datetime
#%matplotlib inline

# This method is done for you. Make sure to put your credentials in twitter.cfg.
def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter


In [2]:
twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [3]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)
 

In [4]:
#import ssl
#import requests.packages.urllib3
#
'''
https://twitter.com/uiucbusiness
https://twitter.com/uofifs - facility & services ???
https://twitter.com/thedailyillini - The Independent Student Newspaper at the UofI
'''
# input_string = "University of Illinois, Urbana-Champaign" or "UIUC" or "UI"
# request = robust_request(twitter,'search/tweets',{'q':'MIT','lang':'en', 'count':100})
# for tweet in request:
#     print tweet['text'] 
#     print '\n'

'\nhttps://twitter.com/uiucbusiness\nhttps://twitter.com/uofifs - facility & services ???\nhttps://twitter.com/thedailyillini - The Independent Student Newspaper at the UofI\n'

In [5]:
def pickle_to_file(filename, stream_of_data):
    f = open(filename,'w')
    for line in stream_of_data:
        u = line['text']
        pickle.dump(line,f)
    f.close()
    print "End of saving to file" , filename

In [6]:
#pickle_to_file('tempdatafile.txt',request)

In [7]:
def unpickle_to_file(inputfile,outputfile):
    inf = open(inputfile,'rb')
    outf = open(outputfile,'w')
    output = pickle.load(inf)

    for key in output:
        if type(output[key]) == dict:
            topKey = output[key]
            outf.write ("%s" % (key + ":"))
            outf.write("\n")
            for k2 in topKey:
                outf.write("   %-30s" % (k2 + ": "))
                outf.write(str(topKey[k2]))
                outf.write("\n")
        else:
            outf.write("%-30s" % (key + ": "))
            outf.write(str(output[key]))
            outf.write("\n")
    outf.close()
    inf.close()
    print "End of writing to file " , outputfile

In [10]:
def get_followers():
    followers = []
    f = open ('iit_followers.txt','rb')
    regexp = re.compile(r'p\d+')
    for line in f:
        if regexp.search(str(line)) is not None:
            continue
        else: 
            if line.startswith('aV'):
                line = line[2:]
                followers.append(line.replace("\n",""))
    followers.sort()
    return followers

followers_list = get_followers()
for follower in followers_list:
    print follower 

007d9ad8eb5c437
02_ankit
047_yash
074819755221326
0BrunoPereira
0c8f4857a9ef4b6
0dpmartin0
100100058Monica
100tomates
101grads
10_west
1111NS1111
115f8aa204c5430
11thWardChicago
123srik
130108021akr
150394alok
150NRiverside
151258D
16786Raju
1755Kamal
18thPilot
1920samarth82
1997_marmar
19febRituraj
1_open
1_state
1armedGeek
1b0bfb7be2514af
1bdec72c2fee4d5
1cE29147
1wordchrisjones
2010cys7095
2011kumar786
2013SAGA
2015_Cubbies
23f487edfaa244d
2609Mahi
2COMS_Career
2batmanrn
2bikes1trailer
2ndPresChicago
3024e5acd13e4b8
3108smit
312Parks
317500
31stStreetBus
3827vt
384663
3DStudentDesign
3b83eeab796c486
3d8768b4501c402
3db2311adff24c9
3dcampustours
3rdAndKyleLong
419pankaj
440CHO
45rules
465811466
4D_arch
4TheRustedNail
4medanish
50DomainTools
543d72a9e4d6494
6122312
6328Shaksa
64thAirRefuelSq
68redvwbug
703fa5c860434ef
71pinto
777VJ
8083867533
86rishabh
8822899670
88adiYoga
8Katalyst8
8himanshu8
8sanjeev2011
8ultraman8
912dheerajYadav
925inMotion
93910875311
96gupta
97a6d254604e466
984

In [11]:
# for follower in followers_list:
#     print follower
print followers_list.index('ASTMStudentFans')

132


In [158]:
def print_tweets(outputfile,follower_index):
    tweets = 0
    f = open(outputfile,'a')
    print 'opened file named' , outputfile
    followers = get_followers()[follower_index:]
    for follower in followers:
        request = twitter.request('statuses/user_timeline',{'screen_name': follower})
        if request.status_code == 401:
            print 'Unauthorized data access. The user does not have access to such information'
            continue
        else:
            if tweets <= 5000:
                f.write(follower + ":")
                for tweet in request:
                    if tweet is not None:
                        f.write(tweet['text'])
            else:
                timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
                newfile = 'outputfile' + timestamp + '.txt'
                print 'calling new file name', newfile
                print_tweets(newfile,followers.index(follower))

outputfile = 'outputfile'+ datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+ '.txt'
#print_tweets(outputfile,0)

In [146]:
print tweets

defaultdict(<type 'list'>, {})


In [260]:
def recurse_dict(dictionary):
    for key in dictionary:
        if type(key) == dict:
            recurse_dict(key)
        else:
            print ("%s" % (key))
            #print dictionary[key]

In [271]:
def rtrv_student_from_twtrpage():
    request = robust_request(twitter, 'users/search', {'q':'TheDailyIllini'})
    for tweet in request:
        print ("----------------------------------------------------------------------")
        for key in tweet:
            print (key + ":")
            print tweet[key]
            print ('\n')

In [1]:
#rtrv_student_from_twtrpage()